# TM10007 Assignment template

In [3]:
# Run this to use from colab environment
!pip install -q --upgrade git+https://github.com/LauraSchellekens/TM10007_-private_-group.git

ERROR: Command errored out with exit status 128: git clone -q https://github.com/LauraSchellekens/TM10007_-private_-group.git /tmp/pip-req-build-b6jsaqug Check the logs for full command output.


## Data loading and cleaning

Below are functions to load the dataset of your choice. After that, it is all up to you to create and evaluate a classification method. Beware, there may be missing values in these datasets. Good luck!

In [4]:
# Data loading functions. Uncomment the one you want to use
from adni.load_data import load_data
#from brats.load_data import load_data
#from hn.load_data import load_data

data = load_data()
print(f'The number of samples: {len(data.index)}')
print(f'The number of columns: {len(data.columns)}')


ModuleNotFoundError: ignored